# Table of contents

1. [Looping through multiple periods of time](#introduction)
    2. [Completing a monthly loop](#paragraph1)
    3. [Completing a weekly loop](#paragraph2)
4. [`Zip()`'ing it all together](#paragraph3)
5. [Utilizing `ExcelWriter()` to export your file](#paragraph4)

## Looping through multiple periods of time <a name="introduction"></a>

You are given a task to prepare quick summary statistics for each month over several years.

A simple loop can solve this and utilizing the `zip()` command will help you generate a well-prepared dataframe to work with.

We'll be using the [superstore dataset](https://github.com/kn-kn/setup-py-functions/blob/master/Sample_Superstore_Data.xls) for our example.

Lets assume you've been asked to produce a single file with two different sheets. The date range will be the order date of January 1, 2012 - December 31, 2012. Each sheet contains different information:

* **Sheet 1**: Contains **monthly results** of: total profit and number of unique customers

* **Sheet 2**: The same as above, but on a **weekly basis** instead. The week starts on January 1, 2012 and is every 7 days from that point on. If the last week is less than 7 days, thats fine

In [1]:
import pandas as pd

In [2]:
# Import Data
df = pd.read_excel("Sample_Superstore_Data.xls")

In [3]:
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2013-152156,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2013-138688,2013-06-13,2013-06-17,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2012-108966,2012-10-11,2012-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2011-110422,2011-01-22,2011-01-24,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2014-121258,2014-02-27,2014-03-04,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2014-121258,2014-02-27,2014-03-04,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2014-121258,2014-02-27,2014-03-04,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


Lets setup the dates as required.

In [4]:
start = pd.Timestamp('2012-01-01')
end = pd.Timestamp('2012-12-31')

You can utilize Pandas `pandas.date_range()` function to generate a list of dates based on your parameters. I'll create one for example that creates a list of the months from the two date variables we just created:

In [5]:
# Freq = 'M' refers to month
dr_month = pd.date_range(start, end, freq='M')

In [6]:
dr_month

DatetimeIndex(['2012-01-31', '2012-02-29', '2012-03-31', '2012-04-30',
               '2012-05-31', '2012-06-30', '2012-07-31', '2012-08-31',
               '2012-09-30', '2012-10-31', '2012-11-30', '2012-12-31'],
              dtype='datetime64[ns]', freq='M')

We've now generated a list of the last day of each month. Lets create one for each week as well then utilize a simple Python loop to loop each of the dates and calculate what we need.

In [7]:
# Freq = 'W' refers to week
dr_week = pd.date_range(start, end, freq='W')

In [8]:
dr_week

DatetimeIndex(['2012-01-01', '2012-01-08', '2012-01-15', '2012-01-22',
               '2012-01-29', '2012-02-05', '2012-02-12', '2012-02-19',
               '2012-02-26', '2012-03-04', '2012-03-11', '2012-03-18',
               '2012-03-25', '2012-04-01', '2012-04-08', '2012-04-15',
               '2012-04-22', '2012-04-29', '2012-05-06', '2012-05-13',
               '2012-05-20', '2012-05-27', '2012-06-03', '2012-06-10',
               '2012-06-17', '2012-06-24', '2012-07-01', '2012-07-08',
               '2012-07-15', '2012-07-22', '2012-07-29', '2012-08-05',
               '2012-08-12', '2012-08-19', '2012-08-26', '2012-09-02',
               '2012-09-09', '2012-09-16', '2012-09-23', '2012-09-30',
               '2012-10-07', '2012-10-14', '2012-10-21', '2012-10-28',
               '2012-11-04', '2012-11-11', '2012-11-18', '2012-11-25',
               '2012-12-02', '2012-12-09', '2012-12-16', '2012-12-23',
               '2012-12-30'],
              dtype='datetime64[ns]', freq='W-S

To calculate the first of each month based on a date you have, utilize `pandas.offsets.Monthbegin()`. See my example below:

In [9]:
from pandas.tseries.offsets import MonthBegin

In [10]:
for after_date in dr_month:
    start_date = after_date.floor('d') - pd.offsets.MonthBegin(1)
    
    # print to see the loop in action
    print(start_date)
    print(after_date)
    print()

2012-01-01 00:00:00
2012-01-31 00:00:00

2012-02-01 00:00:00
2012-02-29 00:00:00

2012-03-01 00:00:00
2012-03-31 00:00:00

2012-04-01 00:00:00
2012-04-30 00:00:00

2012-05-01 00:00:00
2012-05-31 00:00:00

2012-06-01 00:00:00
2012-06-30 00:00:00

2012-07-01 00:00:00
2012-07-31 00:00:00

2012-08-01 00:00:00
2012-08-31 00:00:00

2012-09-01 00:00:00
2012-09-30 00:00:00

2012-10-01 00:00:00
2012-10-31 00:00:00

2012-11-01 00:00:00
2012-11-30 00:00:00

2012-12-01 00:00:00
2012-12-31 00:00:00



To calculate on a weekly basis, we utilize `pandas.Timedelta('1 W')` to generate a day one week after our list of dates. See the example:

In [11]:
for after_date in dr_week:
    start_date = start_date + pd.Timedelta('1 W')
    
    # print to see the loop in action
    print(start_date)
    print(after_date)
    print()

2012-12-08 00:00:00
2012-01-01 00:00:00

2012-12-15 00:00:00
2012-01-08 00:00:00

2012-12-22 00:00:00
2012-01-15 00:00:00

2012-12-29 00:00:00
2012-01-22 00:00:00

2013-01-05 00:00:00
2012-01-29 00:00:00

2013-01-12 00:00:00
2012-02-05 00:00:00

2013-01-19 00:00:00
2012-02-12 00:00:00

2013-01-26 00:00:00
2012-02-19 00:00:00

2013-02-02 00:00:00
2012-02-26 00:00:00

2013-02-09 00:00:00
2012-03-04 00:00:00

2013-02-16 00:00:00
2012-03-11 00:00:00

2013-02-23 00:00:00
2012-03-18 00:00:00

2013-03-02 00:00:00
2012-03-25 00:00:00

2013-03-09 00:00:00
2012-04-01 00:00:00

2013-03-16 00:00:00
2012-04-08 00:00:00

2013-03-23 00:00:00
2012-04-15 00:00:00

2013-03-30 00:00:00
2012-04-22 00:00:00

2013-04-06 00:00:00
2012-04-29 00:00:00

2013-04-13 00:00:00
2012-05-06 00:00:00

2013-04-20 00:00:00
2012-05-13 00:00:00

2013-04-27 00:00:00
2012-05-20 00:00:00

2013-05-04 00:00:00
2012-05-27 00:00:00

2013-05-11 00:00:00
2012-06-03 00:00:00

2013-05-18 00:00:00
2012-06-10 00:00:00

2013-05-25 00:00

Now we have working loops! Plugging in your calculations is as simple as running them in a non-looped situation. Lets create the code we need to answer our request, then plug them into the loop after.

**Request 1: Calculate total profit**

In [12]:
df['Profit'].sum()

286397.0216999999

**Request 2: Calculate number of unique customers**

In [13]:
df['Customer ID'].nunique()

793

Now that we have our code, we have one remaining problem, where do we store the results of each loop? We can store them in a list and utilize `zip()` later on to create our dataframe. Lets generate a bunch of blank lists to start with:

In [14]:
# Create lists for our month loop
list_profit_month = []
list_unique_month = []

# Create lists for our week loop
list_profit_week = []
list_unique_week = []

Remember that we will need to filter our main `df` dataframe for each set of days in the loop. Let's create them!

We will append the lists above with our values once we've calculated them.

## Completing a monthly loop <a name="paragraph1"></a>

In [15]:
for after_date in dr_month:
    start_date = after_date.floor('d') - pd.offsets.MonthBegin(1)
    
    # print to see the loop in action (sanity check)
    print(start_date)
    print(after_date)
    print()
    
    # Generate a filtered dataframe for each loop to work on
    df_filtered = df[(df['Order Date'] >= start_date) & (df['Order Date'] <= after_date)]
    
    # Calculate total profit
    profit = df_filtered['Profit'].sum()
    list_profit_month.append(profit)
    
    # Calculate number of unique customers
    unique_customers = df_filtered['Customer ID'].nunique()
    list_unique_month.append(unique_customers)

2012-01-01 00:00:00
2012-01-31 00:00:00

2012-02-01 00:00:00
2012-02-29 00:00:00

2012-03-01 00:00:00
2012-03-31 00:00:00

2012-04-01 00:00:00
2012-04-30 00:00:00

2012-05-01 00:00:00
2012-05-31 00:00:00

2012-06-01 00:00:00
2012-06-30 00:00:00

2012-07-01 00:00:00
2012-07-31 00:00:00

2012-08-01 00:00:00
2012-08-31 00:00:00

2012-09-01 00:00:00
2012-09-30 00:00:00

2012-10-01 00:00:00
2012-10-31 00:00:00

2012-11-01 00:00:00
2012-11-30 00:00:00

2012-12-01 00:00:00
2012-12-31 00:00:00



Lets take a look what are in the lists

In [16]:
list_profit_month

[-3281.007000000001,
 2821.2761999999993,
 9724.6724,
 4187.4962,
 4667.868999999999,
 3335.5571999999997,
 3288.648299999999,
 5355.808399999999,
 8209.162699999997,
 2817.366,
 12474.788399999998,
 8016.9658999999965]

In [17]:
list_unique_month

[28, 38, 76, 69, 69, 68, 64, 64, 126, 79, 146, 141]

Finally lets repeat but on a weekly basis. The code is almost the same as the requirements are very similar:

## Completing a weekly loop <a name="paragraph2"></a>

In [18]:
dr_week

DatetimeIndex(['2012-01-01', '2012-01-08', '2012-01-15', '2012-01-22',
               '2012-01-29', '2012-02-05', '2012-02-12', '2012-02-19',
               '2012-02-26', '2012-03-04', '2012-03-11', '2012-03-18',
               '2012-03-25', '2012-04-01', '2012-04-08', '2012-04-15',
               '2012-04-22', '2012-04-29', '2012-05-06', '2012-05-13',
               '2012-05-20', '2012-05-27', '2012-06-03', '2012-06-10',
               '2012-06-17', '2012-06-24', '2012-07-01', '2012-07-08',
               '2012-07-15', '2012-07-22', '2012-07-29', '2012-08-05',
               '2012-08-12', '2012-08-19', '2012-08-26', '2012-09-02',
               '2012-09-09', '2012-09-16', '2012-09-23', '2012-09-30',
               '2012-10-07', '2012-10-14', '2012-10-21', '2012-10-28',
               '2012-11-04', '2012-11-11', '2012-11-18', '2012-11-25',
               '2012-12-02', '2012-12-09', '2012-12-16', '2012-12-23',
               '2012-12-30'],
              dtype='datetime64[ns]', freq='W-S

In [19]:
for start_date in dr_week:
    after_date = start_date + pd.Timedelta('1 W')
    
    # print to see the loop in action
    print(start_date)
    print(after_date)
    print()
    
    # Generate a filtered dataframe for each loop to work on
    df_filtered = df[(df['Order Date'] >= start_date) & (df['Order Date'] <= after_date)]
    
    # Calculate total profit
    profit = df_filtered['Profit'].sum()
    list_profit_week.append(profit)
    
    # Calculate number of unique customers
    unique_customers = df_filtered['Customer ID'].nunique()
    list_unique_week.append(unique_customers)

2012-01-01 00:00:00
2012-01-08 00:00:00

2012-01-08 00:00:00
2012-01-15 00:00:00

2012-01-15 00:00:00
2012-01-22 00:00:00

2012-01-22 00:00:00
2012-01-29 00:00:00

2012-01-29 00:00:00
2012-02-05 00:00:00

2012-02-05 00:00:00
2012-02-12 00:00:00

2012-02-12 00:00:00
2012-02-19 00:00:00

2012-02-19 00:00:00
2012-02-26 00:00:00

2012-02-26 00:00:00
2012-03-04 00:00:00

2012-03-04 00:00:00
2012-03-11 00:00:00

2012-03-11 00:00:00
2012-03-18 00:00:00

2012-03-18 00:00:00
2012-03-25 00:00:00

2012-03-25 00:00:00
2012-04-01 00:00:00

2012-04-01 00:00:00
2012-04-08 00:00:00

2012-04-08 00:00:00
2012-04-15 00:00:00

2012-04-15 00:00:00
2012-04-22 00:00:00

2012-04-22 00:00:00
2012-04-29 00:00:00

2012-04-29 00:00:00
2012-05-06 00:00:00

2012-05-06 00:00:00
2012-05-13 00:00:00

2012-05-13 00:00:00
2012-05-20 00:00:00

2012-05-20 00:00:00
2012-05-27 00:00:00

2012-05-27 00:00:00
2012-06-03 00:00:00

2012-06-03 00:00:00
2012-06-10 00:00:00

2012-06-10 00:00:00
2012-06-17 00:00:00

2012-06-17 00:00

## `Zip()`'ing it all together <a name="paragraph3"></a>

Now we have 2 things completed:

* A list of dates
* Lists of our completed calculations

To put everything together in a dataframe, we can utilize `zip()`!

The function will look something like this:

`pd.DataFrame(zip(), index=, columns=[])`

The parameters you need to fill in include the following:

* **`zip()`:** Add in all the lists here, separated by commas
* **`index=[]`:** This is your index; we will use the list of dates we have here
* **`columns=[]`:** Whatever you want your column names to be

In [20]:
df_final_month = pd.DataFrame(zip(list_profit_month, list_unique_month), index=dr_month,
                              columns=['Profit', '# of Unique Customers'])

In [21]:
df_final_month

,Profit,# of Unique Customers
2012-01-31,-3281.0070,28
2012-02-29,2821.2762,38
2012-03-31,9724.6724,76
2012-04-30,4187.4962,69
2012-05-31,4667.8690,69
2012-06-30,3335.5572,68
2012-07-31,3288.6483,64
2012-08-31,5355.8084,64
2012-09-30,8209.1627,126
2012-10-31,2817.3660,79


Now repeat the same for the weeks!

In [22]:
df_final_week = pd.DataFrame(zip(list_profit_week, list_unique_week), index=dr_week,
                              columns=['Profit', '# of Unique Customers'])

In [23]:
df_final_week

,Profit,# of Unique Customers
2012-01-01,-1334.9318,10
2012-01-08,-19.1117,6
2012-01-15,-321.7616,4
2012-01-22,-1946.9618,6
2012-01-29,499.1717,8
2012-02-05,485.6351,12
2012-02-12,688.2032,8
2012-02-19,1414.5440,7
2012-02-26,299.7378,11
2012-03-04,996.8737,17


## Utilizing `ExcelWriter()` to export your file <a name="paragraph4"></a>

Now we can export our two dataframes into a single Excel file!

In [24]:
writer = pd.ExcelWriter("Summary_Results.xlsx", engine="xlsxwriter")
df_final_month.to_excel(writer, sheet_name="Monthly Results")
df_final_week.to_excel(writer, sheet_name="Weekly Results")

writer.save()